# Training Pipeline

## Continued Pretraining

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/Dash-7b/Dash-7b

In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 14.0 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login
login()

In [ ]:
!python pretrain.py \
    --model_type llama \
    --model_name_or_path meta-llama/Llama-2-7b-chat-hf \
    --train_file_dir ./data/pretrain \
    --validation_file_dir ./data/pretrain \
    --per_device_train_batch_size 2 \
    --per_device_eval_batch_size 2 \
    --do_train \
    --do_eval \
    --use_peft True \
    --seed 42 \
    --max_train_samples 10000 \
    --max_eval_samples 10 \
    --num_train_epochs 1 \
    --learning_rate 2e-4 \
    --warmup_ratio 0.05 \
    --weight_decay 0.01 \
    --logging_strategy steps \
    --logging_steps 10 \
    --eval_steps 50 \
    --fp16 False \
    --evaluation_strategy steps \
    --save_steps 500 \
    --save_strategy steps \
    --save_total_limit 3 \
    --gradient_accumulation_steps 1 \
    --preprocessing_num_workers 1 \
    --block_size 1024 \
    --output_dir outputs-pt-v1 \
    --overwrite_output_dir \
    --ddp_timeout 30000 \
    --logging_first_step True \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype float16 \
    --device_map auto \
    --report_to tensorboard \
    --ddp_find_unused_parameters False \
    --gradient_checkpointing True

2023-12-18 17:00:38.103706: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-18 17:00:38.103771: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-18 17:00:38.105423: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-18 17:00:39.367800: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-12-18 17:00:39.918 | INFO     | __main__:main:354 - Model args: ModelArguments(model_type='llama', model_name_or_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_name_or_path=None, load_in_8bit=False, load_in_4bit=False, cache_dir=None, 

In [ ]:
%ls -lh outputs-pt

total 0


**Merge Weights**

In [ ]:
!tensorboard --logdir output_dir/runs --host 0.0.0.0 --port 8009

2023-12-18 16:02:20.624670: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-18 16:02:20.679703: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-18 16:02:20.679767: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-18 16:02:20.681449: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-18 16:02:20.689535: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
!python merge_peft_adapter.py --model_type llama \
    --base_model meta-llama/Llama-2-7b-chat-hf --lora_model outputs-pt-v1 --output_dir merged-pt/

Namespace(model_type='llama', base_model='meta-llama/Llama-2-7b-chat-hf', tokenizer_path=None, lora_model='outputs-pt-v1', resize_emb=False, output_dir='merged-pt/')
Base model: meta-llama/Llama-2-7b-chat-hf
LoRA model: outputs-pt-v1
Loading LoRA for causal language model
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Loading checkpoint shards: 100% 2/2 [00:04<00:00,  2.03s/it]
Merging with merge_and_unload...
Saving to Hugging Face format...
Done! model saved to merged-pt/


In [ ]:
%ls -lh merged-pt/

total 13G
-rw------- 1 root root  661 Dec 18 16:08 config.json
-rw------- 1 root root  183 Dec 18 16:08 generation_config.json
-rw------- 1 root root 4.6G Dec 18 16:08 pytorch_model-00001-of-00003.bin
-rw------- 1 root root 4.7G Dec 18 16:08 pytorch_model-00002-of-00003.bin
-rw------- 1 root root 3.4G Dec 18 16:09 pytorch_model-00003-of-00003.bin
-rw------- 1 root root  24K Dec 18 16:09 pytorch_model.bin.index.json
-rw------- 1 root root  414 Dec 18 16:08 special_tokens_map.json
-rw------- 1 root root 1.8K Dec 18 16:08 tokenizer_config.json
-rw------- 1 root root 489K Dec 18 16:08 tokenizer.model


In [ ]:
%cat merged-pt/config.json

{
  "_name_or_path": "meta-llama/Llama-2-7b-chat-hf",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.35.2",
  "use_cache": true,
  "vocab_size": 32000
}


## Supervised Fine-tuning

In [ ]:
%ls ./data/finetune

In [ ]:
!python supervised_finetune.py \
    --model_type llama \
    --model_name_or_path merged-pt \
    --train_file_dir ./data/finetune \
    --validation_file_dir ./data/finetune \
    --per_device_train_batch_size 4 \
    --per_device_eval_batch_size 4 \
    --do_train \
    --do_eval \
    --use_peft True \
    --max_train_samples 1000 \
    --max_eval_samples 10 \
    --num_train_epochs 1 \
    --learning_rate 2e-5 \
    --warmup_ratio 0.05 \
    --weight_decay 0.05 \
    --logging_strategy steps \
    --logging_steps 10 \
    --eval_steps 50 \
    --evaluation_strategy steps \
    --save_steps 500 \
    --save_strategy steps \
    --save_total_limit 3 \
    --gradient_accumulation_steps 1 \
    --preprocessing_num_workers 1 \
    --output_dir outputs-sft-v1 \
    --overwrite_output_dir \
    --ddp_timeout 30000 \
    --logging_first_step True \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype float16 \
    --device_map auto \
    --report_to tensorboard \
    --ddp_find_unused_parameters False \
    --gradient_checkpointing True

2023-12-18 16:20:13.431063: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-18 16:20:13.431121: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-18 16:20:13.432876: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-18 16:20:14.760648: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-12-18 16:20:15.308 | WARNING  | __main__:__post_init__:209 - You may set max_train_samples = -1 to run all samples in production.
2023-12-18 16:20:15.341 | INFO     | __main__:main:914 - Model args: ModelArguments(model_type='llama', mo

In [ ]:
%ls -lh outputs-sft-v1

total 39M
-rw------- 1 root root  640 Dec 18 16:23 adapter_config.json
-rw------- 1 root root  39M Dec 18 16:23 adapter_model.safetensors
-rw------- 1 root root  348 Dec 18 16:23 all_results.json
-rw------- 1 root root  191 Dec 18 16:23 eval_results.json
-rw------- 1 root root 5.0K Dec 18 16:23 README.md
drwx------ 4 root root 4.0K Dec 18 16:20 runs/
-rw------- 1 root root  438 Dec 18 16:23 special_tokens_map.json
-rw------- 1 root root 1.9K Dec 18 16:23 tokenizer_config.json
-rw------- 1 root root 489K Dec 18 16:23 tokenizer.model
-rw------- 1 root root 4.2K Dec 18 16:23 trainer_state.json
-rw------- 1 root root  177 Dec 18 16:23 train_results.json


In [ ]:
!python merge_peft_adapter.py --model_type llama \
    --base_model merged-pt --lora_model outputs-sft-v1 --output_dir ./merged-sft

Namespace(model_type='llama', base_model='merged-pt', tokenizer_path=None, lora_model='outputs-sft-v1', resize_emb=False, output_dir='./merged-sft')
Base model: merged-pt
LoRA model: outputs-sft-v1
Loading LoRA for causal language model
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Loading checkpoint shards: 100% 3/3 [00:16<00:00,  5.63s/it]
Merging with merge_and_unload...
Saving to Hugging Face format...
Done! model saved to ./merged-sft


In [ ]:
%ls -lh merged-sft/

total 13G
-rw------- 1 root root  641 Dec 18 16:25 config.json
-rw------- 1 root root  183 Dec 18 16:25 generation_config.json
-rw------- 1 root root 4.6G Dec 18 16:25 pytorch_model-00001-of-00003.bin
-rw------- 1 root root 4.7G Dec 18 16:26 pytorch_model-00002-of-00003.bin
-rw------- 1 root root 3.4G Dec 18 16:26 pytorch_model-00003-of-00003.bin
-rw------- 1 root root  24K Dec 18 16:26 pytorch_model.bin.index.json
-rw------- 1 root root  414 Dec 18 16:25 special_tokens_map.json
-rw------- 1 root root 1.8K Dec 18 16:25 tokenizer_config.json
-rw------- 1 root root 489K Dec 18 16:25 tokenizer.model


In [ ]:
%cat merged-sft/config.json

{
  "_name_or_path": "merged-pt",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.35.2",
  "use_cache": true,
  "vocab_size": 32000
}


## Test

In [ ]:
!python inference.py --model_type llama --base_model merged-sft --interactive